In [1]:
import ee
import pandas as pd

In [2]:
import ee
ee.Authenticate()
# service_account = 'prajwalkhanal@modislaidownload.iam.gserviceaccount.com'
# credentials = ee.ServiceAccountCredentials(service_account, '/home/khanalp/modislaidownload-06fa52f24bc6.json')
# ee.Initialize(credentials)

Enter verification code:  4/1AeaYSHD9wW3aTs_KVV6a6L7RBdFm0368scyis1MVZbseTTdhg7OSLNtkVHA



Successfully saved authorization token.


In [3]:
ee.Initialize()

In [4]:
stations = pd.read_excel("Stationdetails.xlsx")

In [5]:
stations

,station_name,start_year,end_year,Name,Country,"Position (lon,lat)",Site type (IGBP),Elevation above sea
0,BE-Bra,1996,2020,Brasschaat,Belgium (BE),"4.51984,51.30761",evergreen needleleaf forests,160 (m)
1,BE-Dor,2011,2020,Dorinne,Belgium (BE),"4.968113,50.311874",grasslands,2530
2,BE-Lon,2004,2020,Lonzee,Belgium (BE),"4.746234,50.55162",croplands,1700
3,BE-Maa,2016,2020,Maasmechelen,Belgium (BE),"5.631851,50.97987",closed shrublands,870
4,BE-Vie,1996,2020,Vielsalm,Belgium (BE),"5.998099,50.304962",mixed forests,4900
...,...,...,...,...,...,...,...,...
63,CH-Dav,1997,2020,Davos,Switzerland (CH),"9.85591,46.81533",evergreen needleleaf forests,NaN
64,CH-Aws,2006,2020,Alp Weissenstein,Switzerland (CH),"9.790417,46.583194",grasslands,NaN
65,CH-Cha,2005,2020,Chamau,Switzerland (CH),"8.410444,47.210222",grasslands,393
66,CH-Fru,2005,2020,"Fr¸eb¸el, Kt. Zug",Switzerland (CH),"8.537778 ,47.115833",grasslands,982


In [6]:
stations.rename(columns={'Position (lon,lat)': 'Position'}, inplace=True)

In [7]:
# Initialize an empty dictionary to store station_name, latitude, and longitude
station_coordinates = {}

# Iterate over each row in the DataFrame
for index, row in stations.iterrows():
    # Extract the station_name and position
    station_name = row['station_name']
    position = row['Position']
    
    # Split the position string to get latitude and longitude
    longitude, latitude = map(float, position.split(','))
    
    # Store the station_name, latitude, and longitude in the dictionary
    station_coordinates[station_name] = {'latitude': latitude, 'longitude': longitude}


In [8]:
len(station_coordinates)

68

In [9]:
# Define the date range
start_date = '2000-01-01'
end_date = '2020-12-31'

In [10]:
# Function to export images for a specific station
def export_image_for_station(image, station_name,buffered_point):
    # Get the timestamp from system:time_start property
    timestamp = ee.Date(image.get('system:time_start')).format('yyyyMMdd').getInfo()
    
    # Define the folder based on the station name
    folder_name = 'MODIS_LAI_Export/' + station_name
    
    # Export the image to Google Drive
    task = ee.batch.Export.image.toDrive(image=image,
                                         description='MODIS_LAI_' + station_name + '_' + timestamp,
                                         folder=folder_name,
                                         fileNamePrefix='MODIS_LAI_' + station_name + '_' + timestamp,
                                         region=buffered_point.bounds(),
                                         scale=500,
                                         crs='EPSG:4326')
    task.start()

In [11]:
subset_stations = {k: station_coordinates[k] for k in list(station_coordinates)[1:6]}

In [12]:
subset_stations

{'BE-Dor': {'latitude': 50.311874, 'longitude': 4.968113},
 'BE-Lon': {'latitude': 50.55162, 'longitude': 4.746234},
 'BE-Maa': {'latitude': 50.97987, 'longitude': 5.631851},
 'BE-Vie': {'latitude': 50.304962, 'longitude': 5.998099},
 'CZ-BK1': {'latitude': 49.502075, 'longitude': 18.536882}}

In [ ]:
# Iterate over each station in the dictionary
for station_name, coordinates in subset_stations.items():
    # Extract latitude and longitude for the station
    latitude = coordinates['latitude']
    longitude = coordinates['longitude']
    
    # Define the location (longitude, latitude)
    point = ee.Geometry.Point(longitude, latitude)
    
    # Define a buffer of 500m around the point
    buffered_point = point.buffer(1000)
    
    # Create an image collection for MODIS LAI data
    modis_collection = ee.ImageCollection('MODIS/006/MOD15A2H') \
        .filterBounds(buffered_point) \
        .filterDate(ee.Date(start_date), ee.Date(end_date)) \
        .select(['Lai_500m', 'FparLai_QC', 'LaiStdDev_500m'])  # Add other bands too
    
    # Export all images in the collection for the current station
    image_list = modis_collection.toList(modis_collection.size())
    for i in range(image_list.size().getInfo()):
        export_image_for_station(ee.Image(image_list.get(i)), station_name,buffered_point)
        
    


**Steps to do**
1) Read all tif files in the xarray.
2) Then, select the grid cell containing particular location and neighboring 8 grid cells.
3) Keep only good-quality data (QC flag values0, 2, 24, 26, 32, 34, 56, and 58) and set all other values to missing.
4) At each time step, calculate a weighted mean from the nine pixels by weighting them by their standard deviation error (defined as $1/\sigma^2$).
5)  Gap fill the resulting 8-daily time series using a cubic spline function (Forsythe et al., 1977) and set any negative LAI values set to zero.
6)  The gap-filled time series was first smoothed using a cubic smoothing spline.
7)  A climatology (46 time steps) was then calculated from all available years.
8)  An anomaly time series was then created by removing the climatology and smoothed by taking a rolling mean over a window of ±6 time steps to further remove short-term variability.
9)  The climatology was then added to the smoothed time series and the 8-daily time series interpolated to the time resolution of
the flux tower data using the climatological values prior to MODIS commencing in January 2000.